In [10]:
# Imports

import torch
from torchvision import models, transforms
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display, clear_output
from jetbot import Camera, bgr8_to_jpeg
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import math
from jetbot import Robot
import utils
import threading
import map_utils
import sys

In [7]:
import traitlets
import numpy as np
from jetbot import bgr8_to_jpeg

class MapProvider2(traitlets.HasTraits):
    value = traitlets.Bytes()

    def __init__(self, map_size):
        super().__init__()
        self.map_size = map_size
        # Poniżej przykładowe inicjalne wartości (możesz podmienić na swoje)
        self.robot_x = map_size / 2
        self.robot_y = map_size / 2
        self.map_grid = np.zeros((self.map_size, self.map_size), dtype=int)
        self._color_map = {
            -1: [169, 169, 169],  # Szary - nieznane
             0: [0, 255, 0],      # Zielony - podłoga
             1: [255, 250, 205],  # Bardzo jasnożółty
             2: [255, 215, 0],    # Żółty
             3: [255, 165, 0],    # Pomarańczowy
             4: [255, 140, 0],    # Ciemnopomarańczowy
             5: [255, 0, 0]       # Czerwony - przeszkoda
        }
        self.lut = np.array([
            [169, 169, 169],  # index 0 => dla map_grid == -1
            [0,   255, 0],    # index 1 => dla map_grid ==  0
            [255, 250, 205],  # index 2 => dla map_grid ==  1
            [255, 215, 0],    # index 3 => dla map_grid ==  2
            [255, 165, 0],    # index 4 => dla map_grid ==  3
            [255, 140, 0],    # index 5 => dla map_grid ==  4
            [255, 0,   0],    # index 6 => dla map_grid ==  5
        ], dtype=np.uint8)
        
        # Tworzymy początkowy obraz (wartości w self.value)
        self.update_map()
        
    def _draw_robot_circle(self, img, radius=3, color=(255, 0, 0)):
        height, width, _ = img.shape
        for dy in range(-radius, radius + 1):
            for dx in range(-radius, radius + 1):
                # obliczamy odległość euklidesową od środka
                if dx*dx + dy*dy <= radius*radius:
                    px = int(self.robot_x) + dx
                    py = int(self.robot_y) + dy
                    # sprawdzamy, czy punkt mieści się w granicach obrazka
                    if 0 <= px < width and 0 <= py < height:
                        img[py, px] = color

    def update_map(self):
        # 1) Obliczamy indeksy do tablicy LUT
        indices = self.map_grid + 1  # -1 -> 0, 0 -> 1, ..., 5 -> 6
        # 2) Pobieramy kolory w trybie wektorowym
        img = self.lut[indices]      # shape => (map_size, map_size, 3)
        # 3) Rysujemy pozycję robota (bez OpenCV)
        self._draw_robot_circle(img)
        # 4) Odwracamy pionowo, jeśli chcesz mieć (0,0) na dole
        img = np.flipud(img)
        # 5) Kodujemy do JPEG
        self.value = bgr8_to_jpeg(img)

    def set_map(self, map_grid):
        self.map_grid = map_grid
        self.update_map()
    
    def set_robot_pos(self, x, y):
        self.robot_x = x
        self.robot_y = y
        self.update_map()


In [22]:
# Robot
robot = Robot()

# Parametry sterowania
FORWARD_SPEED = 0.5
TURN_SPEED = 0.5
DELAY = 0.1     # Czas odświeżania pętli w sekundach
MAP_SIZE = 400  # Rozmiar mapy
CELL_SIZE = 0.01  # 1 pixel = 0.01 m
ROBOT_SIZE = 0.2  # Rozmiar robota w metrach
VISION_RANGE = 0.2  # Jak daleko widzi robot (0.2m = 20 pikseli)
FORWARD_SPEED_MS = FORWARD_SPEED * 0.16  # Prędkość w metrach na sekundę
TURN_ANGLE = 30  # Robot skręca o 30 stopni przy losowym wyborze kierunku

# Inicjalizacja mapy (-1 = nieznane)
map_grid = np.full((MAP_SIZE, MAP_SIZE), -1)

# Pozycja początkowa robota w środku mapy
robot_x, robot_y = MAP_SIZE // 2, MAP_SIZE // 2
robot_direction = 0  # Kąt obrotu w stopniach (0 = góra)

In [4]:
# Model rozpoznawania podłogi
floor_prototype = np.load("../models/floor_prototype_full.npy")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()  # Usunięcie ostatniej warstwy
model = model.to(device)
model.eval()  # Tryb ewaluacji

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Widget kamera 
camera = Camera.instance(width=300, height=300)
image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [23]:
# Widget Mapa
map_provider = MapProvider2(map_size=MAP_SIZE)
map_widget = widgets.Image(format='jpeg', width=300, height=300)
map_link = traitlets.dlink((map_provider, 'value'), (map_widget, 'value'))

map_provider.set_robot_pos(robot_x, robot_y)
map_provider.set_map(map_grid)

display(map_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [18]:
def move_and_print(distance=sys.maxsize):
    global robot_direction, robot_x, robot_y
    last_move_time = time.time()  # Czas ostatniego ruchu do przodu
    previous_move_was_forward = False  # Czy ostatni ruch był do przodu?
    try:
        #print(f"START X: {robot_x}    Y:{robot_y}")
        has_turned = False  # Flaga: Czy robot ostatnio skręcał?
        turn_direction = None  # Kierunek ostatniego skrętu: 'left' lub 'right'

        # while True
        for step in range(distance):
            print(f"STEP {step:4d}: X: {robot_x:6.2f}, Y: {robot_y:6.2f}, direction: {robot_direction:6.2f}")
            image = Image.fromarray(camera.value)
            forward_distance = utils.is_floor(image, model, device, floor_prototype)
            #utils.update_map(forward_distance, robot_x, robot_y, robot_direction, map_grid)
            #map_provider.set_robot_pos(robot_x, robot_y)
            #map_provider.set_map(map_grid)

            if previous_move_was_forward:
                current_time = time.time()
                time_elapsed = current_time - last_move_time
                last_move_time = current_time  # Aktualizacja ostatniego ruchu
                step_size = FORWARD_SPEED_MS * time_elapsed / CELL_SIZE
                # Obliczenie nowej pozycji robota
                angle_rad = np.radians(robot_direction)
                #robot_x += int(step_size * np.cos(angle_rad))
                #robot_y += int(step_size * np.sin(angle_rad))
                robot_y += step_size * np.cos(angle_rad)
                robot_x += step_size * np.sin(angle_rad)

            if forward_distance < 0.175: #and not utils.any_red_20_20(map_grid, robot_direction, robot_x, robot_y):  # Jeśli podłoga wykryta
                #print(f"Podłoga wykryta. Jedź do przodu. Odległość: {forward_distance:.4f}")
                utils.update_map(forward_distance, int(robot_x), int(robot_y), robot_direction, map_grid)
                utils.move_forward(robot, FORWARD_SPEED)
                last_move_time = time.time()
                time.sleep(DELAY)  # Jedź chwilę do przodu
                has_turned = False
                turn_direction = None
                previous_move_was_forward = True
            else:
                #print(f"Brak podłogi przed robotem. Odległość: {forward_distance:.4f}")
                previous_move_was_forward = False
                utils.stop(robot)
                if has_turned:
                    pass
                    #print(f"Kontynuowanie skrętu w kierunku: {turn_direction}")
                else:
                    # Rozejrzenie się i wybór najlepszego kierunku
                    #print("Rozejrzenie się...")
                    utils.turn_by_angle(robot, -TURN_ANGLE/2, TURN_SPEED)  # Obrót w lewo o 15°
                    image = Image.fromarray(camera.value)
                    left_distance = utils.is_floor(image, model, device, floor_prototype)
                    utils.turn_by_angle(robot, TURN_ANGLE, TURN_SPEED)  # Obrót w prawo o 30° (15° w prawo od pozycji wyjściowej)
                    image = Image.fromarray(camera.value)
                    right_distance = utils.is_floor(image, model, device, floor_prototype)
                    # Powrót do pozycji początkowej (15° w lewo od pozycji prawej)
                    utils.turn_by_angle(robot, -TURN_ANGLE/2, TURN_SPEED)
                    # Wybór kierunku
                    if left_distance < right_distance:
                        #print(f"Skręt w lewo o {TURN_ANGLE}°. Odległość w lewo: {left_distance:.4f}")
                        turn_direction = 'left'
                        forward_distance = left_distance
                    else:
                        #print(f"Skręt w prawo o {TURN_ANGLE}°. Odległość w prawo: {right_distance:.4f}")
                        turn_direction = 'right'
                        forward_distance = right_distance
                    has_turned = True

                if turn_direction == 'left':
                    utils.update_map(forward_distance, int(robot_x), int(robot_y), robot_direction, map_grid)
                    utils.turn_by_angle(robot, -TURN_ANGLE, TURN_SPEED)
                    robot_direction = (robot_direction - TURN_ANGLE) % 360
                elif turn_direction == 'right':
                    utils.update_map(forward_distance, int(robot_x), int(robot_y), robot_direction, map_grid)
                    utils.turn_by_angle(robot, TURN_ANGLE, TURN_SPEED)
                    robot_direction = (robot_direction + TURN_ANGLE) % 360
            
            #if i % 10 ==0:
            map_provider.set_robot_pos(robot_x, robot_y)
            map_provider.set_map(map_grid)

    except KeyboardInterrupt:
        print("Zatrzymano robota.")
    finally:
        utils.stop(robot)

    


In [28]:
move_and_print(40)

STEP    0: X: 266.85, Y: 260.28, direction:   0.00
STEP    1: X: 266.85, Y: 260.28, direction:   0.00
STEP    2: X: 266.85, Y: 261.89, direction:   0.00
STEP    3: X: 266.85, Y: 263.47, direction:   0.00
STEP    4: X: 266.85, Y: 264.92, direction:   0.00
STEP    5: X: 266.85, Y: 266.35, direction:   0.00
STEP    6: X: 266.85, Y: 267.79, direction:   0.00
STEP    7: X: 266.85, Y: 269.21, direction:   0.00
STEP    8: X: 266.85, Y: 270.65, direction:   0.00
STEP    9: X: 266.85, Y: 272.07, direction:   0.00
STEP   10: X: 266.85, Y: 273.56, direction:   0.00
STEP   11: X: 266.85, Y: 274.95, direction:   0.00
STEP   12: X: 266.85, Y: 276.38, direction:   0.00
STEP   13: X: 266.85, Y: 277.80, direction:   0.00
STEP   14: X: 266.85, Y: 279.24, direction:   0.00
STEP   15: X: 266.85, Y: 280.62, direction:   0.00
STEP   16: X: 266.85, Y: 282.04, direction:   0.00
STEP   17: X: 266.85, Y: 283.52, direction:   0.00
STEP   18: X: 266.85, Y: 284.96, direction:   0.00
STEP   19: X: 266.85, Y: 286.35

In [27]:
#utils.draw_map(robot_x.astype(int), robot_y.astype(int), map_grid.astype(int), MAP_SIZE)

In [ ]:
camera.stop()